In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
df_train_raw = pd.read_csv('../train/data/train.csv')
df_test_raw = pd.read_csv('../train/data/test.csv')

df_train_raw.columns

/tmp/ipykernel_33845/1642445820.py:1: DtypeWarning: Columns (101) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_raw = pd.read_csv('../train/data/train.csv')


Index(['wagnum', 'month', 'target_month', 'target_day', '10d_nasip_mean',
       '5d_avg_distance_min', '10d_isload_mean', '60d_opor_station_sign_mean',
       '60d_skoroport_mean', '60d_avg_distance_max',
       ...
       'kuzov', 'telega', 'tormoz', 'tipvozd', 'tippogl', 'norma_km',
       'ownertype', 'expected_srok_sl_y', 'expected_srok_sl_y_b',
       'date_build_b'],
      dtype='object', length=114)

In [75]:
numerical = df_train_raw.select_dtypes(exclude=['object']).columns

corr = df_train_raw[numerical].corr()

print(corr['target_month'].sort_values(ascending=False)[:10])
print(corr['target_month'].sort_values(ascending=True)[:10])

target_month          1.000000
target_day            0.559757
days_from_last_rem    0.165221
wagnum                0.090729
rod_id_x              0.070751
rod_id_y              0.070725
60d_nasip_mean        0.059957
60d_naval_mean        0.059957
30d_naval_mean        0.058540
30d_nasip_mean        0.058540
Name: target_month, dtype: float64
ost_prob               -0.245094
days_to_pl_rem         -0.167966
tara                   -0.068950
cnsi_volumek           -0.068342
expected_srok_sl_y     -0.066991
telega                 -0.049758
30d_another_road_max   -0.041021
10d_another_road_max   -0.034080
cnsi_probeg_dr         -0.032811
5d_another_road_max    -0.030679
Name: target_month, dtype: float64


In [76]:
import re

targets = ['target_month', 'target_day']
numerical = list(filter(lambda x: re.match(r'\d+[dm]_', x), df_train_raw.columns))
categorical = ['kod_vrab', 'model',  'zavod_build', 'kuzov', 'telega', 'expected_srok_sl_y_b', 'date_build_b', 'manage_type', 'rod_id_x', 'reestr_state',
                'tormoz', 'tipvozd', 'tippogl', 'ownertype']
dates = ['month', 'date_kap', 'date_dep', 'date_pl_rem', 'last_rem_date', 'date_build', 'srok_sl']

numerical += [
    "days_to_pl_rem",
    "days_from_last_rem",
    "ost_prob",
    "gruz",
    "cnsi_gruz_capacity",
    "cnsi_volumek",
    "tara",
    "cnsi_probeg_dr",
    "cnsi_probeg_kr",
    "norma_km",
    "expected_srok_sl_y"
]

notna_features = ['date_kap', 'date_dep', 'date_iskl']
notna_features_res = [feature + '_notna' for feature in notna_features]
categorical += notna_features_res

print('Unused columns:')
for col in df_train_raw.columns:
    if col not in targets and col not in numerical and col not in categorical and col not in dates:
        print(col)

features = numerical + categorical + targets
train_features = numerical + categorical

medians = df_train_raw[numerical].median()

def preprcoess_df(df, test=False):
    for date in dates:
        df[date] = pd.to_datetime(df[date])
    df = df[df['month'] > '2022-08-01']

    for col in notna_features:
        df[col + '_notna'] = df[col].notna().astype(int)

    df[categorical] = df[categorical].astype('str')

    return df

df_train = preprcoess_df(df_train_raw)
df_test = preprcoess_df(df_test_raw, test=True)


df_train.shape, df_test.shape

Unused columns:
wagnum
ownership_type
rod_id_y
date_iskl


/tmp/ipykernel_33845/1609641170.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + '_notna'] = df[col].notna().astype(int)
/tmp/ipykernel_33845/1609641170.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + '_notna'] = df[col].notna().astype(int)
/tmp/ipykernel_33845/1609641170.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

((203610, 117), (33707, 117))

In [77]:
df_train.head()

,wagnum,month,target_month,target_day,10d_nasip_mean,5d_avg_distance_min,10d_isload_mean,60d_opor_station_sign_mean,60d_skoroport_mean,60d_avg_distance_max,...,tipvozd,tippogl,norma_km,ownertype,expected_srok_sl_y,expected_srok_sl_y_b,date_build_b,date_kap_notna,date_dep_notna,date_iskl_notna
33975,0,2022-09-01,0,0,0.0,1980.115725,0.0,NaN,NaN,NaN,...,1,11.0,160000,0,32.484932,long,"(0, 2000]",1,1,0
33976,1,2022-09-01,0,0,0.0,2138.510870,1.0,NaN,NaN,NaN,...,4,1.0,160000,0,31.682192,long,"(0, 2000]",1,1,0
33977,2,2022-09-01,0,0,0.8,1920.025345,0.7,NaN,NaN,NaN,...,1,15.0,110000,0,32.065753,long,"(0, 2000]",1,1,0
33978,3,2022-09-01,0,0,0.0,1920.025345,0.0,NaN,NaN,NaN,...,4,1.0,110000,0,32.019178,long,"(0, 2000]",1,1,0
33979,4,2022-09-01,0,0,0.0,1917.051546,0.0,NaN,NaN,NaN,...,6,1.0,160000,0,34.186301,long,"(0, 2000]",1,1,0


In [79]:
df_train[df_train.wagnum == 101][['60d_diff_mean', '30d_diff_mean', '5d_diff_mean', 'target_month', 'ost_prob',]]

,60d_diff_mean,30d_diff_mean,5d_diff_mean,target_month,ost_prob
34076,NaN,146.966667,52.4,0,35094.0
68052,191.40,235.833333,192.6,1,28019.0
102028,151.25,73.466667,0.0,0,157931.0
136004,142.05,210.633333,486.4,0,151612.0
169995,149.05,75.533333,0.0,0,148988.0
203975,105.30,153.533333,73.0,0,144382.0


In [80]:
medians = df_train[numerical].median()

def fill_nans(df):
    df['date_iskl'] = df['date_iskl'].fillna(pd.to_datetime('2025-01-01'))
    df['date_iskl'] = pd.to_datetime(df['date_iskl'])
    df['days_from_last_rem'] = df['days_from_last_rem'].fillna(1200)
    df[numerical] = df[numerical].fillna(medians)
    return df

df_train = fill_nans(df_train)
df_test = fill_nans(df_test)

/tmp/ipykernel_33845/111068538.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_iskl'] = df['date_iskl'].fillna(pd.to_datetime('2025-01-01'))
/tmp/ipykernel_33845/111068538.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_iskl'] = pd.to_datetime(df['date_iskl'])
/tmp/ipykernel_33845/111068538.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [82]:
def add_features(df):
    df['days_to_iskl'] = (df['date_iskl'] - df['month']).dt.days
    df['days_to_srok_sl'] = (df['srok_sl'] - df['month']).dt.days
    df['iskl_in_a_year'] = df['days_to_iskl'] < 365
    df['ost_prob_in_a_month'] = df['ost_prob'] - df['30d_diff_mean'] * 30

    return df


df_train = add_features(df_train)
df_test = add_features(df_test)

/tmp/ipykernel_33845/2549205113.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_to_iskl'] = (df['date_iskl'] - df['month']).dt.days
/tmp/ipykernel_33845/2549205113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_to_srok_sl'] = (df['srok_sl'] - df['month']).dt.days
/tmp/ipykernel_33845/2549205113.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [83]:
numerical = df_train_raw.select_dtypes(exclude=['object']).columns

corr = df_train_raw[numerical].corr()

print(corr['target_month'].sort_values(ascending=False)[:10])
print(corr['target_month'].sort_values(ascending=True)[:10])

target_month          1.000000
target_day            0.559757
days_from_last_rem    0.165221
wagnum                0.090729
rod_id_x              0.070751
rod_id_y              0.070725
60d_naval_mean        0.059957
60d_nasip_mean        0.059957
30d_naval_mean        0.058540
30d_nasip_mean        0.058540
Name: target_month, dtype: float64
ost_prob               -0.245094
days_to_pl_rem         -0.167966
tara                   -0.068950
cnsi_volumek           -0.068342
expected_srok_sl_y     -0.066991
telega                 -0.049758
date_pl_rem            -0.048325
30d_another_road_max   -0.041021
10d_another_road_max   -0.034080
cnsi_probeg_dr         -0.032811
Name: target_month, dtype: float64


In [84]:
df_train.to_csv('../train/data/train.csv', index=False)
df_test.to_csv('../train/data/test.csv', index=False)